<a href="https://colab.research.google.com/github/olusegvn/colab-notebooks/blob/main/English_to_German_translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.3.0

     |████████████████████████████████| 320.4MB 42kB/s 
     |████████████████████████████████| 460kB 44.3MB/s 
     |████████████████████████████████| 20.1MB 7.6MB/s 
     |████████████████████████████████| 2.9MB 35.1MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Found existing installation: tensorflow 

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import unicodedata
import re
import os
import json
import sklearn
import numpy as np
from IPython.display import Image
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer,Input, Dense, Flatten, Conv2D, MaxPooling2D

dataset from http://www.manythings.org/anki/ to build the neural translation model. This dataset consists of over 200,000 pairs of sentences in English and German.

#### Import the data

The dataset is available for download as a zip file at the following link:

https://drive.google.com/open?id=1KczOciG7sYY7SB9UlBeRP1T9659b121Q



In [ ]:
# Run this cell to connect to your Drive folder

from google.colab import drive
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/Datasets')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Run this cell to load the dataset

NUM_EXAMPLES = 20000
data_examples = []
with open('deu.txt', 'r', encoding='utf8') as f:
    for line in f.readlines():
        if len(data_examples) < NUM_EXAMPLES:
            data_examples.append(line)
        else:
            break

In [ ]:
# These functions preprocess English and German sentences

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"ü", 'ue', sentence)
    sentence = re.sub(r"ä", 'ae', sentence)
    sentence = re.sub(r"ö", 'oe', sentence)
    sentence = re.sub(r'ß', 'ss', sentence)
    
    sentence = unicode_to_ascii(sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r"[^a-z?.!,']+", " ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    
    return sentence.strip()

The custom model consists of an encoder RNN and a decoder RNN. The encoder takes words of an English sentence as input, and uses a pre-trained word embedding to embed the words into a 128-dimensional space. To indicate the end of the input sentence, a special end token (in the same 128-dimensional space) is passed in as an input. This token is a TensorFlow Variable that is learned in the training phase (unlike the pre-trained word embedding, which is frozen).

The decoder RNN takes the internal state of the encoder network as its initial state. A start token is passed in as the first input, which is embedded using a learned German word embedding. The decoder RNN then makes a prediction for the next German word, which during inference is then passed in as the following input, and this process is repeated until the special `<end>` token is emitted from the decoder.

In [ ]:
f_english = lambda x: preprocess_sentence(re.split('\t', x)[0])   
f_german = lambda x: '<start> '+preprocess_sentence(re.split('\t', x)[1])+' <end>'
english_sentences = list(map(f_english, data_examples))
german_sentences = list(map(f_german, data_examples))


In [ ]:
tokenizer =  tf.keras.preprocessing.text.Tokenizer(filters='')
tokenizer.fit_on_texts(german_sentences)
tokenizer_config = tokenizer.get_config()
word_indices = json.loads(tokenizer_config['word_index'])
index_words = json.loads(tokenizer_config['index_word'])

In [ ]:
german_sentences_seq = tokenizer.texts_to_sequences(german_sentences)

In [ ]:
idx = np.random.choice(len(english_sentences), 5, replace=False)
print(f"Chosen random indices: {idx}\n")
for _ in idx:
  print(english_sentences[_])
  print(german_sentences[_])
  print(list(map(lambda x: word_indices[x], german_sentences[_].split(' '))), end='\n\n')

Chosen random indices: [10816  6580   666 17377 13319]

i'll enjoy this .
<start> das werde ich geniessen . <end>
[1, 11, 39, 4, 2023, 3, 2]

are they alive ?
<start> sind sie am leben ? <end>
[1, 23, 8, 146, 152, 7, 2]

i'm a man .
<start> ich bin ein mann . <end>
[1, 4, 15, 19, 193, 3, 2]

we depend on you .
<start> wir sind auf sie angewiesen . <end>
[1, 17, 23, 29, 8, 2237, 3, 2]

you're arrogant .
<start> ihr seid arrogant . <end>
[1, 27, 73, 1311, 3, 2]



In [ ]:
  german_sentences_seq = tf.keras.preprocessing.sequence.pad_sequences(german_sentences_seq, maxlen=None, padding='post', value=0)

## 2. Prepare the data

In [ ]:
# Load embedding module from Tensorflow Hub

embedding_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1", 
                                 output_shape=[128], input_shape=[], dtype=tf.string)

In [ ]:
# Test the layer

embedding_layer(tf.constant(["these", "aren't", "the", "droids", "you're", "looking", "for"])).shape

TensorShape([7, 128])

In [ ]:
# Create a random training and validation set split of the data, reserving e.g. 20% of the data 
# for validation (NB: each English dataset example is a single sentence string, and each German dataset example is a sequence of padded integer tokens).


# Load the training and validation sets into a tf.data.Dataset object, passing in a tuple of English and German data for both training and validation sets.


x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(english_sentences, german_sentences_seq, test_size=0.2, shuffle=True)

training_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
validation_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
print(training_dataset.element_spec)
print(validation_dataset.element_spec)

(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(14,), dtype=tf.int32, name=None))
(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(14,), dtype=tf.int32, name=None))


In [ ]:
# Create a function to map over the datasets that splits each English sentence at spaces. Apply this function
#  to both Dataset objects using the map method. Hint: look at the tf.strings.split function.

def map_split(x,y):
  return tf.strings.split(x, ' '),  y
training_dataset = training_dataset.map(map_split)
validation_dataset = validation_dataset.map(map_split)

In [ ]:
# Create a function to map over the datasets that embeds each sequence of English words using the loaded embedding layer/model. 
# Apply this function to both Dataset objects using the map method.
def map_embeddings_func(x,y):
  return embedding_layer(x), y
  
training_dataset = training_dataset.map(map_embeddings_func)
validation_dataset = validation_dataset.map(map_embeddings_func)


In [ ]:
# Create a function to filter out dataset examples where the English sentence is greater than or equal to than 13 (embedded) tokens in length.
#  Apply this function to both Dataset objects using the filter method.


def dataset_filter(x, y):
  return len(x) >= tf.constant(13)

training_dataset = training_dataset.filter(dataset_filter)
validation_dataset = validation_dataset.filter(dataset_filter)
print(training_dataset.element_spec)
print(validation_dataset.element_spec)


(TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(14,), dtype=tf.int32, name=None))
(TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(14,), dtype=tf.int32, name=None))


In [ ]:


# Create a function to map over the datasets that pads each English sequence of embeddings with some distinct padding value before the sequence, 
#  so that each sequence is length 13. Apply this function to both Dataset objects using the map method. Hint: look at the tf.pad function. You can extract a Tensor shape using tf.shape; you might also find the tf.math.maximum function useful.


def pad_embeddings(x,y):
  return tf.pad(x,paddings=[[tf.constant(13)-len(x),0],[0,0]], mode='CONSTANT', constant_values=0), y


training_dataset = training_dataset.map(pad_embeddings)
validation_dataset = validation_dataset.map(pad_embeddings)

In [ ]:

# Batch both training and validation Datasets with a batch size of 16.

# Print the element_spec property for the training and validation Datasets.


training_dataset = training_dataset.batch(16)
validation_dataset = validation_dataset.batch(16)

print(training_dataset.element_spec)
print(validation_dataset.element_spec)


(TensorSpec(shape=(None, None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(None, 14), dtype=tf.int32, name=None))
(TensorSpec(shape=(None, None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(None, 14), dtype=tf.int32, name=None))


In [ ]:
# Using the Dataset .take(1) method, print the shape of the English data example from the training Dataset.

# Using the Dataset .take(1) method, print the German data example Tensor from the validation Dataset.

# print(next(iter(training_dataset.take(1)))[0].shape)
# print(next(iter(training_dataset.take(1)))[1].shape)


In [ ]:
# Using layer subclassing, create a custom layer that takes a batch of English data examples from one of the Datasets, 
#  and adds a learned embedded ‘end’ token to the end of each sequence.

# This layer should create a TensorFlow Variable (that will be learned during training) that is 128-dimensional (the size of the embedding space).
#  Hint: you may find it helpful in the call method to use the tf.tile function to replicate the end token embedding across every element in the batch.

# Using the Dataset .take(1) method, extract a batch of English data examples from the training Dataset and print the shape. Test the custom layer by calling the layer on the English data batch Tensor and print the resulting Tensor shape (the layer should increase the sequence length by one).

class MyLayer(Layer):
  def __init__(self, **kwargs):
    super(MyLayer, self).__init__(**kwargs)
    self.learned_end= tf.Variable(initial_value=tf.zeros(shape=(1,128)))
  
  def call(self, inputs):
    self.learned_end = tf.tile(self.learned_end, [tf.shape(inputs)[0],1])
    self.learned_end = tf.expand_dims(self.learned_end, axis=1, name=None)
    return tf.keras.layers.Concatenate(axis=1)([inputs, self.learned_end])


## 4. Build the encoder network


In [ ]:
inputs = Input(batch_shape=(None, 13, 128))
x = MyLayer()(inputs)
x = tf.keras.layers.Masking(mask_value=0, name='masking_layer')(x)
x,hidden_s,cell_s = tf.keras.layers.LSTM(units=512, return_state = True, return_sequences=True, name='LSTM_states')(x)
encoder_model = Model(inputs = inputs, outputs= [hidden_s,cell_s], name = 'encoder')

In [ ]:
# inputs = Input(batch_shape=(None, 13, 128))

# def return_my_encoeder_model():
#   my_layer = MyLayer()
#   x = my_layer(inputs)
#   x = tf.keras.layers.Masking(mask_value=0, name='masking')(x)
#   x,hidden_s,cell_s = tf.keras.layers.LSTM(units=512, return_state = True, name='LSTM_states')(x)
#   del my_layer
#   return Model(inputs = inputs, outputs= [hidden_s,cell_s])

# MyEncoderModel = return_my_encoeder_model()
# MyEncoderModel.summary()

In [ ]:
# batch = next(iter(training_dataset.take(1)))[0]
# print(batch.shape)
# [print(_.shape) for _ in encoder_model(batch)]

In [ ]:
encoder_model.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 13, 128)]         0         
_________________________________________________________________
my_layer (MyLayer)           (None, 14, 128)           0         
_________________________________________________________________
masking_layer (Masking)      (None, 14, 128)           0         
_________________________________________________________________
LSTM_states (LSTM)           [(None, 14, 512), (None,  1312768   
Total params: 1,312,768
Trainable params: 1,312,768
Non-trainable params: 0
_________________________________________________________________


In [ ]:
class Decoder(Model):
  def __init__(self, **kwargs):
    super(Decoder, self).__init__(**kwargs)
    # Define layers
    self.layer_1 = tf.keras.layers.Embedding(input_dim = len(tokenizer.word_index)+1, mask_zero=True, output_dim=128) 
    self.layer_2= tf.keras.layers.LSTM(units=512,return_sequences=True, return_state=True)
    self.layer_3 = Dense(units = len(tokenizer.word_index) + 1)
  
  def call(self, inputs, hidden_state=None, cell_state=None):
    # Define forward pass
    x = self.layer_1(inputs)
    if hidden_state is not None and cell_state is not None:
      x,hidden_s,cell_s = self.layer_2(x,initial_state = [hidden_state,cell_state])
    else:
      x,hidden_s,cell_s = self.layer_2(x)
    x = self.layer_3(x)
    return x,hidden_s,cell_s

In [ ]:
# next(iter(training_dataset.take(1)))

In [ ]:
decoder = Decoder()
# uncomment the following line
# hidden,cell = encoder_model(next(iter(training_dataset.take(1)))[0])

for english_sample, german_example in training_dataset.take(1):
  hidden_state, cell_state = encoder_model(english_sample)
  decoder_output, hidden_state, cell_state = decoder(german_example)

  print(f'encoder state shape: {hidden_state}')
  print(f'decoder output shape: {decoder_output}')
  print(f'cell state shape: {cell_state}')


In [ ]:
# decoder.summary()

## 6. Custom training loop

In [ ]:
def get_german_data(data):
  return german_data[:, :-1], german_data[:, 1:]

In [ ]:
optimizer= tf.keras.optimizers.Adam(learning_rate=0.001)
variables = encoder_model.trainable_variables + decoder.trainable_variables

In [ ]:
@tf.function
def grad_fn(english_inputs, german_inputs, German_outputs):
  with tf.GradientTape() as tape:
    hidden_state, cell_state = encoder_model(english_inputs)
    decoder_outputs, *_ = decoder(german_inputs,hidden_state=hidden_state,cell_state=cell_state)
    loss_value = tf.keras.losses.SparseCategoricalCrossentropy(German_outputs,decoder_outputs, from_logits=True)
    return loss_value, tape.gradient(loss_value, variables)

In [ ]:
  train_loss = []
  val_loss_results = []   
  for epoch in range(5):
    epoch_loss_avg = tf.keras.metrics.Mean()
    val_epoch_loss_avg = tf.keras.metrics.Mean()
    print('a\n\n')
    for english_input, german_data in training_dataset:
      german_input, german_output = get_german_data(german_data)
      loss_value, grads= grad_fn(english_input, german_input, german_output)
      print(loss_value+'\n\n')
      optimizer.apply_gradients(zip(grads, variables))
      epoch_loss_avg(loss_value)
    for english_input, german_data in validation_dataset:
      german_input, german_output = get_german_data(german_data)
      hidden_state, cell_state = encoder_model(english_inptut)
      decoder_output, *_ = decoder_network(german_input, hidden_state=hidden_state, cell_state=cell_state)
      loss = tf.keras.losses.SparseCategoricalCrossentropy(german_output,decoder_outputs, from_logits=True)
      val_epoch_loss_avg(loss)
    train_loss.append(epoch_loss_avg.result())
    val_loss_results.append(val_epoch_loss_avg.result())
    print(f" Epoch :{epoch}: Loss: {train_loss[0]} : Val_Loss: {val_loss_results[0]}")

a




In [ ]:
fig, axis = plt.subplots(1, 2, sharex=True, figsize=(12, 5))
axis[0].set_xlabel('epochs')
axis[0].set_ylabel('Training loss')
axis[0].plot(train_loss_results)

axis[1].set_xlabel('epochs')
axis[1].set_ylabel('Validaion loss')
axis[1].plot(val_loss_results)

## 7. Use the model to translate


In [ ]:
num_samples = 5
inx = np.random.choice(len(english_sentences), num_samples, replace=False)

In [ ]:
for i,Chunk in enumerate(np.array(english_sentences)[inx]):
  result = []
  chunk2 = tf.strings.split(Chunk,' ')
  chunk2 = embedding_layer(chunk2)
  if chunk2.shape[0] >13:
    print("More than 13 embedded Tokens---Skipping")
    continue
  else:
    chunk2 = tf.pad(chunk2,paddings=[[13-tf.shape(chunk2)[0],0],[0,0]], mode='CONSTANT', constant_values=0)
    abs= tf.expand_dims(chunk2, axis=0)
    hidden_stat,cell_stat = encoder_model(chunk2)
    decoder_input = tf.expand_dims([word_indexs.get('<start>')],0)
    for t in range(training_dataset.element_spec[1].shape[1]):
      out,hidden_stat,cell_stat = decoder(decoder_input,hidden_stat,cell_stat)
      [[predicted_id]] = tf.argmax(out, axis=2).numpy()
      result.append(predicted_id)
      if index_words.get(predicted_id) == '<end>':
        break
      # the predicted ID is fed back into the model
      decoder_input = tf.expand_dims([predicted_id], 0)
    result = tokenizer.sequences_to_texts([result])
    print(f"Sample :{i+1} English Sentence: {chunk} German Translation: {result}")